In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/bootcamp/Averiguacao Cadastral/Dados

/content/drive/MyDrive/bootcamp/Averiguacao Cadastral/Dados


In [3]:
!pip install catboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose  import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier

# Classificadores
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [5]:
# Load the datasets
df_familia_pessoas_final = pd.read_csv('df_familia_pessoas_final.csv',nrows=100000)
df_familia_pessoas_final = df_familia_pessoas_final.sample(10000)

In [6]:
df_familia_pessoas_final.columns

Index(['cd_ibge', 'uf_ibge', 'regiao_ibge', 'estrato', 'classf', 'id_familia',
       'vlr_renda_media_fam', 'cod_local_domic_fam', 'cod_especie_domic_fam',
       'qtd_comodos_domic_fam', 'qtd_comodos_dormitorio_fam',
       'cod_material_piso_fam', 'cod_material_domic_fam',
       'cod_agua_canalizada_fam', 'cod_abaste_agua_domic_fam',
       'cod_banheiro_domic_fam', 'cod_escoa_sanitario_domic_fam',
       'cod_destino_lixo_domic_fam', 'cod_iluminacao_domic_fam',
       'cod_calcamento_domic_fam', 'cod_familia_indigena_fam',
       'ind_familia_quilombola_fam', 'ind_parc_mds_fam', 'qtde_pessoas',
       'peso.fam', 'dias_cadastramento', 'dias_atualizacao', 'classe_renda',
       'id_pessoa', 'cod_sexo_pessoa', 'idade', 'cod_parentesco_rf_pessoa',
       'cod_raca_cor_pessoa', 'cod_local_nascimento_pessoa',
       'cod_certidao_registrada_pessoa', 'cod_deficiencia_memb',
       'cod_sabe_ler_escrever_memb', 'ind_frequenta_escola_memb',
       'cod_escola_local_memb', 'cod_curso_frequ

In [7]:
# Selecionando as variáveis que serão utilizadas para o modelo
X = df_familia_pessoas_final [['dias_cadastramento', 'dias_atualizacao','cod_familia_indigena_fam', 'ind_familia_quilombola_fam','ind_parc_mds_fam',
                                'qtd_pessoas', 'infancia', 'crianca_adolescente', 'adultos', 'idoso', 'deficiencia']]

y = df_familia_pessoas_final['classe_renda']

# Identificando features numéricas e categóricas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns


# Criando pipelines para transformar as features
# Added a 'passthrough' step to the numeric transformer. This allows the numeric features to be included
# without any transformation. Remove this or modify with relevant preprocessing steps if necessary
numeric_transformer = Pipeline(steps=[
    ('passthrough', 'passthrough')
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # Added a name for the step
])

# Combinando os pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Aplicando a transformação aos dados
X_transformed = preprocessor.fit_transform(X)

In [8]:
# Criando os modelos individuais
models = {
#    'LogisticReg': LogisticRegression(),
#    'DecisionTree': DecisionTreeClassifier(max_depth=7, min_samples_split=5),
    'RandomForest': RandomForestClassifier(max_depth=10, n_estimators=100),
    'XGBoost': XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=300),
    'CatBoostClassifier': CatBoostClassifier(depth=5, iterations=250, learning_rate=0.1)
}

# Criando o ensemble com votação majoritária
ensemble = VotingClassifier(estimators=list(models.items()), voting='hard')

In [9]:
# Criando os modelos individuais
models = {
#    'LogisticReg': LogisticRegression(),
#    'DecisionTree': DecisionTreeClassifier(max_depth=7, min_samples_split=5),
    'RandomForest': RandomForestClassifier(max_depth=10, n_estimators=100),
    'XGBoost': XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=300),
    'CatBoostClassifier': CatBoostClassifier(depth=5, iterations=250, learning_rate=0.1)
}

# Criando o ensemble com votação majoritária
# Changed voting to 'soft' to handle potential differences in output shape
ensemble = VotingClassifier(estimators=list(models.items()), voting='soft')

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# Treinando o ensemble
ensemble.fit(X_train, y_train)

# Fazendo previsões
y_pred = ensemble.predict(X_test)

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

0:	learn: 1.0341776	total: 69ms	remaining: 17.2s
1:	learn: 0.9856175	total: 86.1ms	remaining: 10.7s
2:	learn: 0.9466164	total: 99.3ms	remaining: 8.18s
3:	learn: 0.9124035	total: 111ms	remaining: 6.82s
4:	learn: 0.8858649	total: 128ms	remaining: 6.26s
5:	learn: 0.8626797	total: 139ms	remaining: 5.64s
6:	learn: 0.8442832	total: 162ms	remaining: 5.63s
7:	learn: 0.8279673	total: 174ms	remaining: 5.26s
8:	learn: 0.8132344	total: 204ms	remaining: 5.46s
9:	learn: 0.8007891	total: 213ms	remaining: 5.12s
10:	learn: 0.7896678	total: 222ms	remaining: 4.82s
11:	learn: 0.7803553	total: 233ms	remaining: 4.62s
12:	learn: 0.7720052	total: 261ms	remaining: 4.76s
13:	learn: 0.7652506	total: 276ms	remaining: 4.66s
14:	learn: 0.7591461	total: 291ms	remaining: 4.56s
15:	learn: 0.7541381	total: 307ms	remaining: 4.49s
16:	learn: 0.7487043	total: 319ms	remaining: 4.38s
17:	learn: 0.7431815	total: 329ms	remaining: 4.24s
18:	learn: 0.7393640	total: 335ms	remaining: 4.07s
19:	learn: 0.7357567	total: 344ms	remain

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier

# Classificadores
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Assumindo que o dataframe df já está carregado e não possui valores ausentes ou variáveis categóricas

# Separando features e target
X = df_familia_pessoas_final [['dias_cadastramento', 'dias_atualizacao','cod_familia_indigena_fam', 'ind_familia_quilombola_fam','ind_parc_mds_fam',
                                'qtd_pessoas', 'infancia', 'crianca_adolescente', 'adultos', 'idoso', 'deficiencia']]
y = df_familia_pessoas_final['classe_renda']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get categorical feature names before preprocessing
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Check if there are categorical features before adding a dummy feature
if not categorical_features:
    # If no categorical features, then cat_features is not needed for CatBoost.
    # Create models dictionary without passing cat_features to CatBoostClassifier.
    models = {
#        'LogisticReg': LogisticRegression(),
#        'DecisionTree': DecisionTreeClassifier(max_depth=7, min_samples_split=5),
        'RandomForest': RandomForestClassifier(max_depth=10, n_estimators=100),
        'XGBoost': XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=300),
        'CatBoostClassifier': CatBoostClassifier(depth=5, iterations=250, learning_rate=0.1)
    }

else:
    # If categorical features are present, pass them to CatBoostClassifier.
    models = {
#        'LogisticReg': LogisticRegression(),
#        'DecisionTree': DecisionTreeClassifier(max_depth=7, min_samples_split=5),
        'RandomForest': RandomForestClassifier(max_depth=10, n_estimators=100),
        'XGBoost': XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=300),
        'CatBoostClassifier': CatBoostClassifier(depth=5, iterations=250, learning_rate=0.1,
                                                   cat_features=categorical_features)
    }

# Criando o ensemble com votação majoritária
# Changed voting to 'soft' to handle potential differences in output shape
ensemble = VotingClassifier(estimators=list(models.items()), voting='soft')

# Treinando o ensemble
ensemble.fit(X_train, y_train)

# Fazendo previsões
y_pred = ensemble.predict(X_test)

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)
print ("Accuracy:", accuracy)



0:	learn: 1.0341776	total: 4.91ms	remaining: 1.22s
1:	learn: 0.9856175	total: 9.6ms	remaining: 1.19s
2:	learn: 0.9466164	total: 13.5ms	remaining: 1.11s
3:	learn: 0.9124035	total: 17.4ms	remaining: 1.07s
4:	learn: 0.8858649	total: 21.3ms	remaining: 1.04s
5:	learn: 0.8626797	total: 25.2ms	remaining: 1.03s
6:	learn: 0.8442832	total: 29.2ms	remaining: 1.01s
7:	learn: 0.8279673	total: 33.1ms	remaining: 1s
8:	learn: 0.8132344	total: 37ms	remaining: 992ms
9:	learn: 0.8007891	total: 40.9ms	remaining: 983ms
10:	learn: 0.7896678	total: 48.1ms	remaining: 1.05s
11:	learn: 0.7803553	total: 56.5ms	remaining: 1.12s
12:	learn: 0.7720052	total: 68.1ms	remaining: 1.24s
13:	learn: 0.7652506	total: 71.9ms	remaining: 1.21s
14:	learn: 0.7591461	total: 75.7ms	remaining: 1.19s
15:	learn: 0.7541381	total: 79.2ms	remaining: 1.16s
16:	learn: 0.7487043	total: 82.9ms	remaining: 1.14s
17:	learn: 0.7431815	total: 86.7ms	remaining: 1.12s
18:	learn: 0.7393640	total: 90.4ms	remaining: 1.1s
19:	learn: 0.7357567	total: 9